In [1]:
import requests
import pandas as pd
from math import ceil
from time import sleep
import time
import json
from IPython.display import display

# 1. Data Retrieval and Integration

In [2]:
BASE = "https://api.openelectricity.org.au/v4"
TOKEN = "oe_3ZPA7phVarPYZc7Qks7uijfE" 
NETWORK = "NEM"     

HEADERS = {
    "Authorization": f"Bearer {TOKEN}",
    "Accept": "application/json",
}


DATE_START = "2025-10-01T00:00:00"  
DATE_END   = "2025-10-08T00:00:00" 


In [3]:
#get all facilities and related units in NEM
def get_nem_facilities():
    url = f"{BASE}/facilities/"
    # According to the documentation: `network_id` may accept an array; `status_id` or `fueltech_id` may also be used for more granular filtering.
    params = {
        "network_id": ["NEM"],          # only NEM
        "status_id": ["operating"],     # only operating
       
    }
    r = requests.get(url, headers=HEADERS, params=params, timeout=60)
    r.raise_for_status()
    js = r.json()
 
    fac_df = pd.json_normalize(js["data"])  # Facilities Layer
    # Expand the units column into rows (record_path=units), including the facility code as meta.
    units_df = pd.json_normalize(
    js["data"],
    record_path="units",
    meta=["code"],
    meta_prefix="facility_",    # ← Prevent duplicate names
    errors="ignore"
    ).rename(columns={"code": "unit_code", "facility_code": "facility_code"})

    # Certain versions list the names as “code” (unit) and “facility_code” (meta). To cover all bases:
    if "unit_code" not in units_df.columns:
        # Common field name: “code” (unit code)
        if "code" in units_df.columns:
            units_df = units_df.rename(columns={"code": "unit_code"})

    return fac_df, units_df

fac_df, units_df = get_nem_facilities()

units_df.head()

,unit_code,fueltech_id,status_id,capacity_registered,capacity_maximum,data_first_seen,data_last_seen,dispatch_type,created_at,updated_at,capacity_storage,emissions_factor_co2,facility_code
0,ADPPV1,solar_utility,operating,24.75,19.00,2021-05-18T13:10:00+10:00,2025-11-07T19:25:00+10:00,GENERATOR,2023-10-18T04:34:30Z,2024-12-16T23:52:12Z,NaN,NaN,ADP
1,ADPPV2,solar_utility,operating,0.20,0.20,NaN,NaN,GENERATOR,2023-10-18T04:34:30Z,2024-12-16T23:50:10Z,NaN,NaN,ADP
2,ADPPV3,solar_utility,operating,0.02,0.02,NaN,NaN,GENERATOR,2023-10-18T04:34:30Z,2024-12-16T23:51:11Z,NaN,NaN,ADP
3,ADPBA1G,battery_discharging,operating,7.76,6.15,2021-05-18T10:55:00+10:00,2025-11-07T23:35:00+10:00,GENERATOR,2023-10-18T04:34:30Z,2025-06-23T05:34:25Z,12.6,NaN,ADP
4,ADPBA1L,battery_charging,operating,7.76,6.15,2021-05-18T09:55:00+10:00,2025-11-07T23:30:00+10:00,LOAD,2023-10-18T04:34:30Z,2025-06-23T05:34:16Z,12.6,NaN,ADP


In [4]:
fac_df.head()

,code,name,network_id,network_region,description,units,updated_at,created_at,location.lat,location.lng
0,ADP,Adelaide Desalination,NEM,SA1,"<p>The Adelaide Desalination plant (ADP), form...","[{'code': 'ADPPV1', 'fueltech_id': 'solar_util...",2025-08-05T06:08:12Z,2023-10-18T04:34:30Z,-35.096948,138.484061
1,ALDGASF,Aldoga,NEM,QLD1,<p>The Aldoga Solar Farm will be approximately...,"[{'code': 'ALDGASF1', 'fueltech_id': 'solar_ut...",2025-03-25T00:52:44Z,2025-01-31T04:19:33Z,-23.839544,151.084900
2,ANGASTON,Angaston,NEM,SA1,<p>Angaston Power Station is a diesel-powered ...,"[{'code': 'ANGAST1', 'fueltech_id': 'distillat...",2025-09-07T01:53:13Z,2023-10-18T04:34:32Z,-34.503948,139.024296
3,APPIN,Appin,NEM,NSW1,"<p>In a world first, EDL developed the largest...","[{'code': 'APPIN', 'fueltech_id': 'gas_wcmg', ...",2025-09-07T01:53:15Z,2023-10-18T04:34:32Z,-34.210868,150.792711
4,ARWF,Ararat,NEM,VIC1,<p>Ararat Wind Farm is wind farm in western Vi...,"[{'code': 'ARWF1', 'fueltech_id': 'wind', 'sta...",2025-07-08T03:42:06Z,2023-10-18T04:34:32Z,-37.263393,143.082116


In [5]:
#get specific facilities time-series data
def fetch_facility_timeseries(facility_codes, date_start=DATE_START, date_end=DATE_END):
    """Request two metrics (power, emissions) for a batch of facility_codes at 5-minute granularity."""
    url = f"{BASE}/data/facilities/{NETWORK}"
    params = {
        "metrics": ["power", "emissions"],
        "interval": "5m",
        "facility_code": facility_codes,  # Batch
        "date_start": date_start,
        "date_end": date_end,
    }
    r = requests.get(url, headers=HEADERS, params=params, timeout=120)
    r.raise_for_status()
    return r.json()

def timeseries_to_df(js):
    """Flatten the return：
       timestamp | unit_code | metric | value | unit
    """
    rows = []
    data_block = js.get("data", [])
    if not isinstance(data_block, list):
        return pd.DataFrame(rows)  # It could be “-”, or simply an empty table.

    for blk in data_block:
        if not isinstance(blk, dict):
            continue
        metric = blk.get("metric")  # 'power' / 'emissions'
        unit   = blk.get("unit")    # 'MW' / 'tCO2e'
        results = blk.get("results", [])
        for res in results:
            if not isinstance(res, dict):
                continue
            # unit_code shall be taken from columns first; if unavailable, it shall be extracted from name.
            unit_code = None
            cols = res.get("columns")
            if isinstance(cols, dict):
                unit_code = cols.get("unit_code")
            name = res.get("name")
            if not unit_code and isinstance(name, str) and "_" in name:
                unit_code = name.split("_", 1)[1]  # e.g. power_ADPBA1 → ADPBA1

            for item in res.get("data", []):  # item = [timestamp, value]
                if isinstance(item, (list, tuple)) and len(item) >= 2:
                    ts, val = item[0], item[1]
                    rows.append({
                        "timestamp": ts,
                        "unit_code": unit_code,
                        "metric": metric,
                        "value": val,
                        "unit": unit
                    })
    return pd.DataFrame(rows)

# First obtain the list of facility_codes (which may be filtered as required by fueltech/network_region to narrow the scope).
facility_codes = fac_df["code"].dropna().unique().tolist()

# Daily quota considerations: Increase the batch size (though this will also result in larger individual responses).
BATCH   = 30
SLEEP_S = 0.25

all_parts = []
req_count = 0

for i in range(0, len(facility_codes), BATCH):
    batch = facility_codes[i:i+BATCH]
    js = fetch_facility_timeseries(batch)
    req_count += 1
    df_part = timeseries_to_df(js)
    all_parts.append(df_part)
    time.sleep(SLEEP_S)

df_unit = pd.concat(all_parts, ignore_index=True) if all_parts else pd.DataFrame()
print("requests used:", req_count, "rows:", len(df_unit))
display(df_unit.head())

requests used: 14 rows: 1958730


,timestamp,unit_code,metric,value,unit
0,2025-10-01T00:00:00+10:00,ADPBA1,power,-0.004,MW
1,2025-10-01T00:05:00+10:00,ADPBA1,power,-0.046,MW
2,2025-10-01T00:10:00+10:00,ADPBA1,power,0.000,MW
3,2025-10-01T00:15:00+10:00,ADPBA1,power,0.003,MW
4,2025-10-01T00:20:00+10:00,ADPBA1,power,-0.018,MW


In [6]:
# Establish a mapping from unit_code to facility_code
if "facility_code" not in df_unit.columns:
    unit_to_fac = units_df[["unit_code", "facility_code"]].dropna().drop_duplicates()
    df_unit = df_unit.merge(unit_to_fac, on="unit_code", how="left")

# Per-facility wide table: One column each for power and emissions 
# at the same point in time and for the same facility (summing across multiple units)
df_fac_wide = (
    df_unit
    .pivot_table(index=["timestamp", "facility_code"], columns="metric", values="value", aggfunc="sum")
    .rename_axis(columns=None)
    .reset_index()
    .sort_values(["timestamp", "facility_code"])
)

# Extract the required columns from fac_df
fac_info = fac_df[["code", "name", "location.lat", "location.lng","network_region"]].rename(
    columns={"code": "facility_code", "name": "facility_name"}
)

# Left join merge (retaining all rows in df_fac_wide)
df_fac_wide = df_fac_wide.merge(fac_info, on="facility_code", how="left")

facility_fuel = (
    units_df[["facility_code", "fueltech_id"]]
    .dropna(subset=["facility_code", "fueltech_id"])
    .groupby("facility_code")["fueltech_id"]
    .agg(lambda s: s.mode().iat[0] if not s.mode().empty else s.iloc[0])
    .reset_index()
)

# Merge fueltech_id into df_fac_wide
df_fac_wide = df_fac_wide.merge(facility_fuel, on="facility_code", how="left")

# Result
display(df_fac_wide.head())


,timestamp,facility_code,emissions,power,facility_name,location.lat,location.lng,network_region,fueltech_id
0,2025-10-01T00:00:00+10:00,0MREH,0.0,0.00,Melbourne A1,-37.661274,144.726302,VIC1,battery
1,2025-10-01T00:00:00+10:00,0MREHA2,0.0,0.00,Melbourne A2,-37.663934,144.726927,VIC1,battery
2,2025-10-01T00:00:00+10:00,0TARONGBESS,0.0,0.00,Tarong,-26.780051,151.912068,QLD1,battery
3,2025-10-01T00:00:00+10:00,0WAMBOWF,0.0,65.23,Wambo,-26.603045,151.246876,QLD1,wind
4,2025-10-01T00:00:00+10:00,ADP,0.0,0.00,Adelaide Desalination,-35.096948,138.484061,SA1,solar_utility


In [7]:
def fetch_market_network(network_region=None,date_start=DATE_START, date_end=DATE_END):
    url = f"{BASE}/market/network/{NETWORK}"
    params = {
        "metrics": ["price", "demand"],
        "interval": "5m",
        "date_start": date_start,
        "date_end": date_end,
        "primary_grouping": "network_region",  # "network" 或 "network_region"
    }
    if network_region:
        params["network_region"] = network_region

    r = requests.get(url, headers=HEADERS, params=params, timeout=60)
    r.raise_for_status()
    return r.json()

# -------- 2) Flattened into a DataFrame (long table: timestamp / metric / value / unit / network_region) --------
def market_to_df(js):
    """
    将 /v4/market/network 返回的数据扁平化为长表：
    列：timestamp | network_region | metric | value | unit
    兼容 columns.region / columns.network_region；必要时从 name 里解析区域。
    """
    rows = []
    data_block = js.get("data", [])
    if not isinstance(data_block, list):
        return pd.DataFrame(rows)

    for blk in data_block:
        metric = blk.get("metric")         # e.g. "price" / "demand"
        unit   = blk.get("unit")           # e.g. "$/MWh" / "MW"

        for res in blk.get("results", []):
            cols = res.get("columns") or {}
            name = res.get("name") or ""

            # Prioritise according to your screenshot: the columns contain {‘region’: ‘NSW1’}
            region = cols.get("region") or cols.get("network_region")

            # Fallback: Parse from the name (e.g., ‘price_NSW1’)
            if region is None and isinstance(name, str):
                m = re.search(r'_(NSW1|VIC1|QLD1|SA1|TAS1)\b', name)
                region = m.group(1) if m else None

            # Expand the chronological points for this region
            for item in res.get("data", []):
                if isinstance(item, (list, tuple)) and len(item) >= 2:
                    ts, val = item[0], item[1]
                    rows.append({
                        "timestamp": ts,
                        "network_region": region,
                        "metric": metric,
                        "value": val,
                        "unit": unit
                    })

    df = pd.DataFrame(rows)
    return df


js = fetch_market_network()

df_market = market_to_df(js)

# # For a wide table (one time + region per row, with price/demand columns):
df_market_wide = (
    df_market
    .pivot_table(
        index=["timestamp", "network_region"],  # One line per region and time point
        columns="metric",                      # Each metric (price, demand) becomes a column
        values="value",                        # Numeric column
        aggfunc="mean"                         # In case of duplicates, take the last one (or alternatively “mean”)
    )
    .rename_axis(columns=None)                 # Remove redundant axis names
    .reset_index()                             # Restore the index to a regular column
    .sort_values(["timestamp", "network_region"])
)
display(df_market_wide.head(10))

,timestamp,network_region,demand,price
0,2025-10-01T00:00:00+10:00,NSW1,7105.57,56.98
1,2025-10-01T00:00:00+10:00,QLD1,5989.24,54.82
2,2025-10-01T00:00:00+10:00,SA1,1564.92,8.11
3,2025-10-01T00:00:00+10:00,TAS1,898.71,0.12
4,2025-10-01T00:00:00+10:00,VIC1,4893.49,8.95
5,2025-10-01T00:05:00+10:00,NSW1,7170.68,80.01
6,2025-10-01T00:05:00+10:00,QLD1,5920.40,67.30
7,2025-10-01T00:05:00+10:00,SA1,1565.38,0.01
8,2025-10-01T00:05:00+10:00,TAS1,897.18,0.20
9,2025-10-01T00:05:00+10:00,VIC1,4889.73,0.01


In [8]:
df_fac_wide = (
    df_fac_wide
    .merge(
        df_market_wide[["timestamp", "network_region", "demand", "price"]],
        on=["timestamp", "network_region"],
        how="left"
    )
)
df_fac_wide = df_fac_wide.rename(columns={"network_region": "market(network_region)"})
df_fac_wide.head()

,timestamp,facility_code,emissions,power,facility_name,location.lat,location.lng,market(network_region),fueltech_id,demand,price
0,2025-10-01T00:00:00+10:00,0MREH,0.0,0.00,Melbourne A1,-37.661274,144.726302,VIC1,battery,4893.49,8.95
1,2025-10-01T00:00:00+10:00,0MREHA2,0.0,0.00,Melbourne A2,-37.663934,144.726927,VIC1,battery,4893.49,8.95
2,2025-10-01T00:00:00+10:00,0TARONGBESS,0.0,0.00,Tarong,-26.780051,151.912068,QLD1,battery,5989.24,54.82
3,2025-10-01T00:00:00+10:00,0WAMBOWF,0.0,65.23,Wambo,-26.603045,151.246876,QLD1,wind,5989.24,54.82
4,2025-10-01T00:00:00+10:00,ADP,0.0,0.00,Adelaide Desalination,-35.096948,138.484061,SA1,solar_utility,1564.92,8.11


# 2. Data Preprocessing

In [9]:
df_fac_wide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668353 entries, 0 to 668352
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   timestamp               668353 non-null  object 
 1   facility_code           668353 non-null  object 
 2   emissions               668353 non-null  float64
 3   power                   668353 non-null  float64
 4   facility_name           668353 non-null  object 
 5   location.lat            668353 non-null  float64
 6   location.lng            668353 non-null  float64
 7   market(network_region)  668353 non-null  object 
 8   fueltech_id             668353 non-null  object 
 9   demand                  667033 non-null  float64
 10  price                   654823 non-null  float64
dtypes: float64(6), object(5)
memory usage: 56.1+ MB


In [10]:
#Convert the time to Australian Eastern Standard Time
df_fac_wide["timestamp"] = pd.to_datetime(df_fac_wide["timestamp"], utc=True)
df_fac_wide["timestamp"] = df_fac_wide["timestamp"].dt.tz_convert("Australia/Sydney")

df_fac_wide["facility_code"]  = df_fac_wide["facility_code"].astype("string")
df_fac_wide["facility_name"]  = df_fac_wide["facility_name"].astype("string")
df_fac_wide["market(network_region)"]  = df_fac_wide["market(network_region)"].astype("string")

In [11]:
neg_demand = df_fac_wide[df_fac_wide["demand"] < 0]
print(f"Negative demand rows: {len(neg_demand)}")
display(neg_demand.head())

Negative demand rows: 290


,timestamp,facility_code,emissions,power,facility_name,location.lat,location.lng,market(network_region),fueltech_id,demand,price
333607,2025-10-04 12:15:00+10:00,ADP,0.0,0.088,Adelaide Desalination,-35.096948,138.484061,SA1,solar_utility,-12.17,-31.19
333608,2025-10-04 12:15:00+10:00,AGLHAL,0.0,0.000,Hallett,-33.349310,138.752633,SA1,gas_ocgt,-12.17,-31.19
333611,2025-10-04 12:15:00+10:00,ANGASTON,0.0,0.000,Angaston,-34.503948,139.024296,SA1,distillate,-12.17,-31.19
333620,2025-10-04 12:15:00+10:00,BARKIPS,0.0,0.000,Barker Inlet,-34.804000,138.524000,SA1,gas_recip,-12.17,-31.19
333632,2025-10-04 12:15:00+10:00,BLUFF,0.0,0.000,The Bluff,-33.367741,138.793790,SA1,wind,-12.17,-31.19


In [12]:
neg_demand = df_fac_wide[df_fac_wide["demand"] < 0]
print(f"Negative demand rows: {len(neg_demand)}")

ratio = (len(neg_demand) / len(df_fac_wide) * 100)
print(f"Negative demand ratio (%): {round(ratio, 2)}")

display(neg_demand.head())

Negative demand rows: 290
Negative demand ratio (%): 0.04


,timestamp,facility_code,emissions,power,facility_name,location.lat,location.lng,market(network_region),fueltech_id,demand,price
333607,2025-10-04 12:15:00+10:00,ADP,0.0,0.088,Adelaide Desalination,-35.096948,138.484061,SA1,solar_utility,-12.17,-31.19
333608,2025-10-04 12:15:00+10:00,AGLHAL,0.0,0.000,Hallett,-33.349310,138.752633,SA1,gas_ocgt,-12.17,-31.19
333611,2025-10-04 12:15:00+10:00,ANGASTON,0.0,0.000,Angaston,-34.503948,139.024296,SA1,distillate,-12.17,-31.19
333620,2025-10-04 12:15:00+10:00,BARKIPS,0.0,0.000,Barker Inlet,-34.804000,138.524000,SA1,gas_recip,-12.17,-31.19
333632,2025-10-04 12:15:00+10:00,BLUFF,0.0,0.000,The Bluff,-33.367741,138.793790,SA1,wind,-12.17,-31.19


In [13]:
neg_price = df_fac_wide[df_fac_wide["price"] < 0]
print(f"Negative price rows: {len(neg_price)}")
display(neg_price.head())

Negative price rows: 279826


,timestamp,facility_code,emissions,power,facility_name,location.lat,location.lng,market(network_region),fueltech_id,demand,price
7260,2025-10-01 01:50:00+10:00,0MREH,0.0,1.4704,Melbourne A1,-37.661274,144.726302,VIC1,battery,4300.21,-0.06
7261,2025-10-01 01:50:00+10:00,0MREHA2,0.0,0.0000,Melbourne A2,-37.663934,144.726927,VIC1,battery,4300.21,-0.06
7264,2025-10-01 01:50:00+10:00,ADP,0.0,0.0000,Adelaide Desalination,-35.096948,138.484061,SA1,solar_utility,1513.45,-0.05
7265,2025-10-01 01:50:00+10:00,AGLHAL,0.0,0.0000,Hallett,-33.349310,138.752633,SA1,gas_ocgt,1513.45,-0.05
7266,2025-10-01 01:50:00+10:00,AGLSOM,0.0,0.0000,Somerton,-37.630949,144.953098,VIC1,gas_ocgt,4300.21,-0.06


In [14]:
df_fac_wide.to_csv("nem_per_facility_power_emissions_5m_2025-10-01_to_08.csv", index=False)

# 3.Data Publishing via MQTT

In [15]:
# MQTT broker settings
import socket

# Unified topic used across publish/subscribe
BROKER_HOST = "test.mosquitto.org" # mqtt broker
BROKER_PORT = 1883                 # mqtt port
TOPIC       = "nem/yjia0057/power_emissions"  # unified topic
LOCAL_TZ    = "Australia/Sydney"               

try:
    s = socket.socket(); s.settimeout(3); s.connect((BROKER_HOST, BROKER_PORT))
    print(f"MQTT broker reachable: {BROKER_HOST}:{BROKER_PORT}")
finally:
    try: s.close()
    except: pass




MQTT broker reachable: test.mosquitto.org:1883


In [16]:
import json, uuid
import paho.mqtt.client as mqtt

# paho v2 first, fallback to v1
def _make_client(cid: str):
    try:
        return mqtt.Client(client_id=cid, protocol=mqtt.MQTTv311, transport="tcp",
                           callback_api_version=mqtt.CallbackAPIVersion.VERSION2)
    except Exception:
        return mqtt.Client(client_id=cid, protocol=mqtt.MQTTv311, transport="tcp")

def start_subscriber(topic=TOPIC, host=BROKER_HOST, port=BROKER_PORT):
     # simple sub, print first 10 messages
    c = _make_client(f"nem-sub-{uuid.uuid4().hex[:6]}")
    cnt = 0
    def on_connect(client, userdata, flags, rc, properties=None):
        print("connected, rc= Success" if rc == 0 else f"rc={rc}")
        client.subscribe(topic, qos=1)
    def on_message(client, userdata, msg):
        nonlocal cnt; cnt += 1
        if cnt <= 10:
            try: print("[sub] sample:", json.loads(msg.payload.decode("utf-8")))
            except: print("[sub] raw:", msg.payload[:160])
    c.on_connect = on_connect; c.on_message = on_message
    c.reconnect_delay_set(1, 10)
    c.connect(host, port, keepalive=60); c.loop_start()
    return c

# avoid duplicate subscriber
try:
    sub.loop_stop(); sub.disconnect()
except:
    pass
sub = start_subscriber()

In [17]:
import time, json, uuid
import pandas as pd

_REGION_CANDS = ["network_region", "market(network_region)", "region"]

 # pick first existing region col
def _detect_region_col(df: pd.DataFrame) -> str:
    for c in _REGION_CANDS:
        if c in df.columns: return c
    raise KeyError("region column not found")

def _best_time_col(df: pd.DataFrame) -> str:
    return "event_time_utc" if "event_time_utc" in df.columns else "timestamp"

 # convert any ts to Sydney local time (+10/+11)
def _to_local_sydney(ts_val):
    ts = pd.to_datetime(ts_val, errors="coerce")
    if pd.isna(ts): 
        return pd.NaT
    if getattr(ts, "tz", None) is None:       
        try:
            return ts.tz_localize(LOCAL_TZ, ambiguous="infer", nonexistent="shift_forward")
        except Exception:
            return ts.tz_localize(LOCAL_TZ)
    else:
        return ts.tz_convert(LOCAL_TZ)
        
# mqtt publisher client
def _connect_client(host=BROKER_HOST, port=BROKER_PORT, user="", password=""):
    try:
        cli = mqtt.Client(client_id=f"nem-pub-{uuid.uuid4().hex[:8]}",
                          protocol=mqtt.MQTTv311, transport="tcp",
                          callback_api_version=mqtt.CallbackAPIVersion.VERSION2)
    except Exception:
        cli = mqtt.Client(client_id=f"nem-pub-{uuid.uuid4().hex[:8]}",
                          protocol=mqtt.MQTTv311, transport="tcp")
    if user: cli.username_pw_set(user, password or "")
    cli.reconnect_delay_set(1, 10)
    try: cli.will_set("nem/system/peer", payload="offline", qos=1, retain=False)
    except: pass
    cli.connect(host, port, keepalive=60)
    return cli

 # light rename
def prepare_for_publish(df: pd.DataFrame) -> pd.DataFrame:
    dfx = df.copy()
    dfx.rename(columns={
        "location.lat": "lat", "location.lng": "lon",
        "latitude": "lat", "longitude": "lon",
        "market(network_region)": "network_region",
    }, inplace=True)
    try:
        if 'df_facilities' in globals():
            meta = df_facilities.copy()
            meta.rename(columns={
                "location.lat": "lat", "location.lng": "lon",
                "latitude": "lat", "longitude": "lon",
                "market(network_region)": "network_region",
            }, inplace=True)
            keep = [c for c in ["facility_code","lat","lon","network_region"] if c in meta.columns]
            if "facility_code" in keep:
                meta = meta[keep].drop_duplicates("facility_code")
                dfx = dfx.merge(meta, on="facility_code", how="left", suffixes=("","_meta"))
                for c in ["lat","lon","network_region"]:
                    if c+"_meta" in dfx.columns:
                        if c in dfx.columns: dfx[c] = dfx[c].combine_first(dfx[c+"_meta"])
                        else: dfx[c] = dfx[c+"_meta"]
                drop_cols = [c for c in dfx.columns if c.endswith("_meta")]
                if drop_cols: dfx.drop(columns=drop_cols, inplace=True)
    except Exception:
        pass
    return dfx

def _row_to_payload(row: pd.Series, region_col: str, seq: int) -> dict:
    tsv   = row.get("event_time_utc", row.get("timestamp"))
    t_loc = _to_local_sydney(tsv)
    ts_iso = t_loc.isoformat() if pd.notna(t_loc) else None

    lat = row.get("lat", row.get("location.lat", row.get("latitude", None)))
    lon = row.get("lon", row.get("lng", row.get("location.lng", row.get("longitude", None))))

    return {
        "timestamp": ts_iso,                 
        "facility_code": row.get("facility_code"),
        "facility_name": row.get("facility_name"),
        "region": row.get(region_col),       
        "lat": lat, "lon": lon,
        "power_mw": row.get("power_mw", row.get("power")),
        "co2_t": row.get("co2_t", row.get("emissions")),
        "price": row.get("price"), "demand": row.get("demand"),
        "seq": seq,  # publish seq
        "fueltech": row.get("fueltech_id")
    }



In [18]:
def publish_from_df_progress(
    df: pd.DataFrame,
    host=BROKER_HOST, port=BROKER_PORT,
    topic=TOPIC, delay=0.1, also_split=True,
    user="", password="", log_every=100,
    skip_no_coords=False
) -> int:
    
    # sort by Sydney local time then facility_code
    dfx  = prepare_for_publish(df)

    tcol = _best_time_col(dfx)
    dfx["__event_local__"] = dfx[tcol].apply(_to_local_sydney)
    dfx = dfx.sort_values(["__event_local__", "facility_code"], kind="mergesort") \
             .drop(columns="__event_local__") \
             .reset_index(drop=True)

    region_col = _detect_region_col(dfx)
    
    # connect and start loop
    cli = _connect_client(host=host, port=port, user=user, password=password)
    cli.loop_start()

    n = len(dfx); t0 = time.time()
    print(f"[publisher] start: total rows = {n}, topic = {topic}, delay = {delay}s")

    for i, row in enumerate(dfx.itertuples(index=False), start=1):
        payload = _row_to_payload(pd.Series(row._asdict()), region_col, i)
        if skip_no_coords and (payload.get("lat") is None or payload.get("lon") is None):
            continue
        j = json.dumps(payload, ensure_ascii=False)
        
        # main topic
        cli.publish(topic, j, qos=1, retain=False)
        time.sleep(float(delay))  # ≥0.1s
        
        # split by region/facility
        if also_split:
            reg = str(payload.get("region", "NA")).replace("/", "-")
            fac = str(payload.get("facility_code", "NA")).replace("/", "-")
            split_topic = f"nem/reg/{reg}/{fac}/power_emissions"
            cli.publish(split_topic, j, qos=1, retain=False)
            time.sleep(float(delay))  # 分 topic 同样等待

        if log_every and i % log_every == 0:
            print(f"[publisher] {i}/{n} ({i/n:5.2%})")

    cli.loop_stop(); cli.disconnect()
    print("[publisher] done.")
    return n

In [19]:
# take 300 rows; final local-time sort is inside publisher
tcol = "event_time_utc" if "event_time_utc" in df_fac_wide.columns else "timestamp"
sample = df_fac_wide.sort_values([tcol, "facility_code"]).head(300).copy()
publish_from_df_progress(
    sample,
    host=BROKER_HOST, port=BROKER_PORT, topic=TOPIC,
    delay=0.1, also_split=True,
    log_every=100,
    skip_no_coords=False
)

connected, rc= Success
[publisher] start: total rows = 300, topic = nem/yjia0057/power_emissions, delay = 0.1s
[sub] sample: {'timestamp': '2025-10-01T00:00:00+10:00', 'facility_code': '0TARONGBESS', 'facility_name': 'Tarong', 'region': 'QLD1', 'lat': -26.780051, 'lon': 151.912068, 'power_mw': 0.0, 'co2_t': 0.0, 'price': 54.82, 'demand': 5989.24, 'seq': 3, 'fueltech': 'battery'}
[sub] sample: {'timestamp': '2025-10-01T00:00:00+10:00', 'facility_code': '0WAMBOWF', 'facility_name': 'Wambo', 'region': 'QLD1', 'lat': -26.603045, 'lon': 151.246876, 'power_mw': 65.23, 'co2_t': 0.0, 'price': 54.82, 'demand': 5989.24, 'seq': 4, 'fueltech': 'wind'}
[sub] sample: {'timestamp': '2025-10-01T00:00:00+10:00', 'facility_code': 'ADP', 'facility_name': 'Adelaide Desalination', 'region': 'SA1', 'lat': -35.096948, 'lon': 138.484061, 'power_mw': 0.0, 'co2_t': 0.0, 'price': 8.11, 'demand': 1564.92, 'seq': 5, 'fueltech': 'solar_utility'}
[sub] sample: {'timestamp': '2025-10-01T00:00:00+10:00', 'facility_cod

300

# 5.Continuous Execution 

In [20]:
# ============================================================
# Task 5 – Continuous Execution Runner
# ============================================================

import time
from datetime import timedelta

def run_continuous_publish(
    df,
    *,
    rounds: int | None = 2,   # How many rounds to run; None = infinite rounds
    round_sleep: int = 60,       # After each round concludes, please wait a further 60 seconds.
    host: str = "test.mosquitto.org",
    port: int = 1883,
    topic: str = "nem/yjia0057/power_emissions",
    delay: float = 0.1,          
    also_split: bool = True,
    log_every: int = 50000,
    skip_no_coords: bool = False,
    date_start: str = DATE_START,
    date_end: str = DATE_END
):
    """
    Task 5：
      - Periodically replay the same data frame to simulate an unbounded data stream
      - Introduce a delay of X seconds between each message
      - After completing each round, 
      wait for Y seconds before restarting the next round from the beginning
    """

    if df is None or len(df) == 0:
        print("[runner] empty df, nothing to publish.")
        return 0

    # Unify time columns and sort
    tcol = "event_time_utc" if "event_time_utc" in df.columns else "timestamp"
    df[tcol] = pd.to_datetime(df[tcol], errors="coerce")
    df = df.dropna(subset=[tcol]).sort_values([tcol, "facility_code"]).reset_index(drop=True)

    # All unique timestamps
    unique_ts = df[tcol].dropna().sort_values().unique()
    n_ts = len(unique_ts)
    if n_ts == 0:
        print("[runner] no valid timestamps found.")
        return 0

    # Window size per round
    window_size = min(300, len(df))
    print(f"[runner] df rows = {len(df)}, unique timestamps = {n_ts}, window_size = {window_size}")

    # Estimating the maximum number of cycles based on the date span (for reset control)
    step_minutes = 5
    span = (pd.to_datetime(date_end) - pd.to_datetime(date_start))
    total_rounds = int(span.total_seconds() // (step_minutes * 60))
    if total_rounds == 0:
        total_rounds = n_ts  # Fallback: Using the number of unique timestamps

    round_idx = 0
    try:
        while True:
            round_idx += 1
            ts_idx = (round_idx - 1) % n_ts
            start_ts = unique_ts[ts_idx]

            # Locate the starting position
            matches = df.index[df[tcol] == start_ts]
            if len(matches) == 0:
                start_pos = 0
            else:
                start_pos = matches[0]

            print(f"\n[runner] round {round_idx}/{total_rounds} start (start_ts = {start_ts}, start_pos = {start_pos})")

            # Retrieve the next 300 rows of data from this point onwards.
            end_pos = min(start_pos + window_size, len(df))
            sample = df.iloc[start_pos:end_pos].copy()
            
            # Play the entire round of df
            sent = publish_from_df_progress(
                sample,
                host=host,
                port=port,
                topic=topic,
                delay=delay,
                also_split=also_split,
                log_every=log_every,
                skip_no_coords=skip_no_coords,
            )
            print(f"[runner] round {round_idx} done: published {sent} rows")
            
            # Determine whether the end has been reached
            if round_idx >= total_rounds:
                print(f"[runner] reached end of dataset after {round_idx} rounds, restarting from beginning in 60s...")
                round_idx = 0  # Reset round
                df = df0.copy()  # Reset data

            # Determine whether the number of rounds has been completed
            if (rounds is not None) and (round_idx >= rounds):
                print("[runner] finished all rounds.")
                break

            # An additional 60-second wait between each wheel
            print(f"[runner] sleeping {round_sleep}s before next round ...")
            time.sleep(round_sleep)

    except KeyboardInterrupt:
        print("\n[runner] stopped by user.")


In [21]:
run_continuous_publish(
    df=df_fac_wide,
    rounds=None,          # None=Infinite Rounds
    round_sleep=60,       # 60 sec
    host=BROKER_HOST,
    port=BROKER_PORT,
    delay=0.1,            # 0.1 sec
    log_every=200,
    skip_no_coords=False,
    date_start = DATE_START,
    date_end = DATE_END
)



[runner] df rows = 668353, unique timestamps = 2016, window_size = 300

[runner] round 1/2016 start (start_ts = 2025-10-01 00:00:00+10:00, start_pos = 0)
[publisher] start: total rows = 300, topic = nem/yjia0057/power_emissions, delay = 0.1s
[publisher] 200/300 (66.67%)
[publisher] done.
[runner] round 1 done: published 300 rows
[runner] sleeping 60s before next round ...

[runner] round 2/2016 start (start_ts = 2025-10-01 00:05:00+10:00, start_pos = 330)
[publisher] start: total rows = 300, topic = nem/yjia0057/power_emissions, delay = 0.1s
[publisher] 200/300 (66.67%)
[publisher] done.
[runner] round 2 done: published 300 rows
[runner] sleeping 60s before next round ...

[runner] round 3/2016 start (start_ts = 2025-10-01 00:10:00+10:00, start_pos = 660)
[publisher] start: total rows = 300, topic = nem/yjia0057/power_emissions, delay = 0.1s
[publisher] 200/300 (66.67%)
[publisher] done.
[runner] round 3 done: published 300 rows
[runner] sleeping 60s before next round ...

[runner] rou